In [41]:
import pandas as pd
import vv
df_list = pd.read_csv(vv.LISTING_BASIC_FILE)
df_indiv = pd.read_csv(vv.LISTING_ENRICHED_FILE)
df_json = pd.read_csv(vv.LISTING_JSONMODEL_FILE)
df_meta = pd.read_csv(vv.LISTING_JSON_META_FILE)

df_json['id'] = df_json['id'].astype(int)
df_meta['id_copy'] = df_meta['id_copy'].astype(int)

HOW = 'left'
df_original = df_list \
    .merge(df_indiv, on='ids', how='inner', suffixes=('', '_listing')) \
    .merge(df_json, left_on='ids',right_on='id', how=HOW, suffixes=('', '_model')) \
    .merge(df_meta, left_on='ids',right_on='id_copy', how=HOW, suffixes=('', '_meta'))

df_original = df_original.rename(index=str, columns={"Station_Prox": "distance to train",'bedrooms_model':'bedrooms','bathrooms_model':'bathrooms'})

# Drop: data protection
df_original = df_original.drop(['Address'],axis=1)

# Drop: only related to data collection
df_original = df_original.drop(['version','version_listing','version_model','referencing_link','date_scraped_model','Links','link'],axis=1)

# Drop: extensive, image, or long text information
df_original = df_original.drop(['keyFeatures','images','industryAffiliations','epcGraphs'],axis=1)

# Drop: Entire element is of no (immediate) use
droppable_column = []
for each in df_original.columns:
    if each.startswith('contactInfo.') \
            or each.startswith('text.') \
            or each.startswith('misInfo.') \
            or each.startswith('date_scrape') \
            or each.startswith('dfpAdInfo.') \
            or (each.startswith('address.') and each not in ['address.outcode'] ) \
            or (each.startswith('location.') and each not in ['location.latitude','location.latitude'])  \
    :
            droppable_column.append(each)

df_original = df_original.drop(droppable_column,axis=1)

# Drop: Duplicate id row
df_original = df_original.drop(['id_copy','id'],axis=1)

# Drop: No information available, it seems
df_original = df_original.drop(['feesApply','lettings','prices.message','countryGuide'],axis=1)
df_original = df_original.drop(['analyticsBranch.branchPostcode','analyticsProperty.selectedPrice','livingCosts.domesticRates','brochures'],axis=1)

df_original.set_index("ids", inplace=True)

df_original.head()

/tmp/ipykernel_5411/3534453798.py:5: DtypeWarning: Columns (23,25,26,28,31,32,40,44,47,51,54,56,59,60,61,62,63,64,74,78,79,80,82,83,84,85,86,88) have mixed types. Specify dtype option on import or set low_memory=False.
  df_json = pd.read_csv(vv.LISTING_JSONMODEL_FILE)


,Description,Price,borough,add_info,distance to train,type,bedrooms,bathrooms,size,other_key_info,...,analyticsProperty.propertyId,analyticsProperty.propertySubType,analyticsProperty.propertyType,analyticsProperty.retirement,analyticsProperty.selectedCurrency,analyticsProperty.soldSTC,analyticsProperty.videoProvider,analyticsProperty.viewType,analyticsProperty.customUri,Unnamed: 46
ids,,,,,,,,,,,,,,,,,,,,,
123171899,3 bedroom apartment for sale,475000.0,"('Enfield', '5E93950')","Added on 09/05/2022 by haart, Southgate",0.6,Apartment,3.0,2.0,NaN,NaN,...,123171899,Apartment,Flats / Apartments,False,NaN,False,Embedded,Current,https://www.rightmove.co.uk/property-for-sale/...,NaN
85461000,3 bedroom terraced house for sale,375000.0,"('Enfield', '5E93950')","Reduced on 17/03/2022 by Christopher Stokes, C...",0.6,Terraced,3.0,1.0,NaN,NaN,...,85461000,Terraced,Houses,False,NaN,False,No Video,Current,https://www.rightmove.co.uk/property-for-sale/...,NaN
85461000,3 bedroom terraced house for sale,375000.0,"('Enfield', '5E93950')","Reduced on 17/03/2022 by Christopher Stokes, C...",0.6,Terraced,3.0,1.0,NaN,NaN,...,85461000,Terraced,Houses,False,NaN,False,No Video,Current,https://www.rightmove.co.uk/property-for-sale/...,NaN
85461000,3 bedroom terraced house for sale,375000.0,"('Enfield', '5E93950')","Reduced on 17/03/2022 by Christopher Stokes, C...",0.6,Terraced,3.0,1.0,NaN,NaN,...,85461000,Terraced,Houses,False,NaN,False,No Video,Current,https://www.rightmove.co.uk/property-for-sale/...,NaN
85461000,3 bedroom terraced house for sale,375000.0,"('Enfield', '5E93950')","Reduced on 17/03/2022 by Christopher Stokes, C...",0.6,Terraced,3.0,1.0,NaN,NaN,...,85461000,Terraced,Houses,False,NaN,False,No Video,Current,https://www.rightmove.co.uk/property-for-sale/...,NaN


In [42]:
df_original.corr()

,distance to train,bedrooms,bathrooms,bedrooms,bathrooms,tenure.yearsRemainingOnLease,livingCosts.annualServiceCharge,analyticsBranch.branchId,analyticsProperty.latitude,analyticsProperty.longitude,analyticsProperty.maxSizeFt,analyticsProperty.minSizeAc,analyticsProperty.minSizeFt
distance to train,1.000000,0.071642,0.036573,0.077775,0.037481,-0.030763,-0.132185,-0.021438,-0.004527,-0.002452,0.006644,0.007676,0.007676
bedrooms,0.071642,1.000000,0.581435,0.999962,0.552152,-0.041717,-0.149710,0.006218,-0.009532,0.003765,0.184226,0.187155,0.187155
bathrooms,0.036573,0.581435,1.000000,0.552074,0.999638,0.044103,0.103970,0.048088,0.000477,-0.027926,0.184475,0.186048,0.186048
bedrooms,0.077775,0.999962,0.552074,1.000000,0.553389,-0.041717,-0.149710,0.003905,-0.009571,0.003195,0.185261,0.188148,0.188148
bathrooms,0.037481,0.552152,0.999638,0.553389,1.000000,0.044103,0.103970,0.046211,0.000602,-0.027548,0.187419,0.188978,0.188978
tenure.yearsRemainingOnLease,-0.030763,-0.041717,0.044103,-0.041717,0.044103,1.000000,-0.466541,0.164734,-0.092003,-0.075277,-0.016513,-0.016513,-0.016513
livingCosts.annualServiceCharge,-0.132185,-0.149710,0.103970,-0.149710,0.103970,-0.466541,1.000000,0.158043,-0.047972,-0.175017,NaN,NaN,NaN
analyticsBranch.branchId,-0.021438,0.006218,0.048088,0.003905,0.046211,0.164734,0.158043,1.000000,0.013814,-0.021817,0.008070,0.009546,0.009546
analyticsProperty.latitude,-0.004527,-0.009532,0.000477,-0.009571,0.000602,-0.092003,-0.047972,0.013814,1.000000,-0.918600,-0.035758,-0.034080,-0.034080
analyticsProperty.longitude,-0.002452,0.003765,-0.027926,0.003195,-0.027548,-0.075277,-0.175017,-0.021817,-0.918600,1.000000,-0.021651,-0.018609,-0.018609
